In [51]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [52]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=chrome_options)

In [53]:
# 무신사 링크
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

In [100]:
# chrome_options = Options()

# chrome_options.add_experimental_option("detach", True)
# chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

# driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=chrome_options)
# # 무신사 링크
# url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
# driver.get(url)
# driver.maximize_window() # 브라우저의 크기 전체화면으로 확대
columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers', 'codimap_explain', 'codimap_hashtag', 'codimap_imgurl']  # 코디맵 카테고리, 제목, 일자, 조회수, 댓글수 크롤링
values = []                                             # 코디맵 정보를 담을 빈 리스트 생성
soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) # 해당 조건에 맞는 정보들을 list로 반환
image_elements = driver.find_elements(By.XPATH, '//img[@class="style-list-thumbnail__img"]')


driver.execute_script("window.scrollTo(0, 700)")
for i, row in enumerate(data_rows):                     # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다

    print('{0}번째 코디맵 크롤링'.format(i+1))
    blank = []                                          # 코디맵 정보를 담을 빈 리스트 생성

    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})       # 코디맵 카테고리 가져오기
    if codimap_category:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        codimap_category = codimap_category.get_text().strip()                                          # html에서 텍스트만 가져온 뒤 공백 제거
        blank.append(codimap_category)
    else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올때 문제발생'.format(i+1))
        continue


    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})      # 제목 가져오기
    if  codimap_title:
        codimap_title =  codimap_title.get_text().strip()
        blank.append( codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 제목 가져올때 문제발생'.format(i+1))
        continue

    all = row.find_all('span', attrs={'class':'post-information__text'})  

    codimap_date = all[0]       # 날짜 가져오기
    if codimap_date:
        codimap_date = codimap_date.get_text().strip()    # class가 Rating__star_area--dFzsb인 span 태그 안에 있는 span 태그에서 텍스트 가져오기
        blank.append(codimap_date)
    else:
        blank.append('Something is wrong')
        print('{}번째 날짜 가져올때 문제발생'.format(i+1))
        continue

    # 조회수 가져오기
    views_element = all[1]
    if views_element:
        views_count = views_element.get_text(strip=True).split()[1]  # "조회 17,219"에서 숫자 부분만 추출
        blank.append(views_count)
    else:
        blank.append('Views count not found')
        print('{}번째 조회수 가져올 때 문제 발생'.format(i+1))
        continue
    


    # 댓글 수 가져오기
    comments_element = all[2]
    if comments_element:
        comment_count = comments_element.get_text(strip=True).split()[1]
        blank.append(comment_count)
    else:
        blank.append('Comments count not found')
        print('{}번째 댓글 수 가져올 때 문제 발생'.format(i+1))
        continue
    # 클릭해서 상세 페이지로 이동 - 클릭
    image_elements[i].click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
    data_row = soup.find('div', attrs={'class':'styling_content codimap-contents'}) # 해당 조건에 맞는 정보들을 list로 반환

    # 코디맵 상세 설명
    codimap_explain = data_row.find('p', attrs={'class':'styling_txt'})       # 코디맵 카테고리 가져오기
    if codimap_explain:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        codimap_explain = codimap_explain.get_text().strip()                                          # html에서 텍스트만 가져온 뒤 공백 제거
        blank.append(codimap_explain)
    else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올때 문제발생'.format(i+1))
        continue


    # 코디맵 해시태그
    codimap_hashtag = data_row.find_all('a', attrs={'class' : 'ui-tag-list__item'})
    insa = []
    for j in range(len(codimap_hashtag)):
        if codimap_explain:
            codimap_hashtag[j] = codimap_hashtag[j].get_text().strip()
            insa.append(codimap_hashtag[j])
            print(codimap_hashtag[j])
        else:                                                                       
            print('{}번째 카테고리 가져올때 문제발생'.format(i+1))
            continue
    str = ''.join(insa)                                    
    blank.append(str)


    # 코디맵 이미지 url
    codimap_imgurl = data_row.find('img', attrs={'class':'photo'})  
    if codimap_imgurl:        
        codimap_imgurl = codimap_imgurl.get('src') 
        codimap_imgurl = 'https:'+codimap_imgurl                      # html에서 텍스트만 가져온 뒤 공백 제거
        blank.append(codimap_imgurl)                                                            # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        
    else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올때 문제발생'.format(i+1))
        continue

    values.append(blank)
    print('---------------------------------------------------')
    driver.back()
    if i == 11:
        driver.execute_script("window.scrollTo(700, 1400)")
    elif i == 23:
        driver.execute_script("window.scrollTo(1400, 2100)")
    elif i == 35:
        driver.execute_script("window.scrollTo(2100, 2800)")
    elif i == 47:
        driver.execute_script("window.scrollTo(2800, 3500)")
    elif i == 59:
        driver.execute_script("window.scrollTo(3500, 4200)")
    elif i == 71:
        driver.execute_script("window.scrollTo(4200, 4900)")
    elif i == 83:
        driver.execute_script("window.scrollTo(4900, 5600)")
    elif i == 95:
        driver.execute_script("window.scrollTo(5600, 6300)")
    elif i == 107:
        driver.execute_script("window.scrollTo(6300, 7000)")
    elif i == 119:
        driver.execute_script("window.scrollTo(7000, 7700)")


 


df = pd.DataFrame(values, columns=columns)
df


1번째 코디맵 크롤링
<li class="style-list-item">
<div class="style-list-item__thumbnail">
<a class="style-list-item__link" href="javascript:void(0);" onclick="goView('4035');" title="톤온톤 코디">
<div class="style-list-thumbnail">
<img alt="톤온톤 코디" class="style-list-thumbnail__img" src="//image.msscdn.net/images/codimap/list/l_3_2021070210342600000002354.jpg?202401101201"/>
</div>
</a>
</div>
<div class="style-list-information">
<a class="style-list-information__link" href="javascript:void(0);" title="톤온톤 코디">
<span class="style-list-information__text" onclick="goCate('casual');">캐주얼</span>
<strong class="style-list-information__title" onclick="goView('4035');">톤온톤 코디</strong>
</a>
</div>
<div class="post-information">
<span class="post-information__text">21.07.02</span>
<span class="post-information__text">조회 17,237</span>
<span class="post-information__text">댓글 50</span>
</div>
</li>
#톤온톤
#뉴트럴톤
#캠퍼스
#피크닉
#와이드핏
#오버핏
#꾸안꾸
#여름
#캐주얼
#벗딥
#엠엠엘지
#캐치볼
#예일
#반원 아틀리에
#토트백
#잡지/무크지
#캡/야구 모자
#캔버스/단화
#반소매 티셔츠


IndexError: list index out of range

In [27]:
# DataFrame을 CSV 파일로 저장
df.to_csv('/Users/daniel/Downloads/codimap_data.csv', index=False, encoding='euc-kr')
# DataFrame을 CSV 파일로 저장 (euc-kr 인코딩 사용)
#df.to_csv('codimap_data.csv', index=False, encoding='euc-kr')
